In [ ]:
# for creating table in SQL

create table RADIOLOGY_IMAGE(
file_path varchar(255),
image_resolution_rows INT, 
image_resolution_columns INT,
CT_slice_thickness FLOAT,
series_type VARCHAR(255),
anatomical_plane VARCHAR(255),
series_serial_number INT,
series_total_number INT,
image_id VARCHAR(255),
series_id VARCHAR(255),
study_id VARCHAR(255),
series_description VARCHAR(3000)
)

create table RADIOLOGY_OCCURRENCE(
person_id varchar(255),
study_date DATE, 
study_datetime DATETIME,
modality VARCHAR(255),
protocol_concept_id VARCHAR(255),
image_total_number INT,
radiology_comment VARCHAR(3000),
study_id VARCHAR(255),
manufacturer VARCHAR(255),
information VARCHAR(3000)
)

In [ ]:
#Run this code at 10.5.99.50 server

In [ ]:
import pyodbc
import pandas as pd
from datetime import datetime

In [ ]:
answer=pd.read_csv('C:/Users/Administrator/Desktop/ChulHyoungPark/answer.csv')
#contrast, PET+CT, SPECT, Functional 있는 건 빼고 매핑할거야
i=answer[answer['LongCommonName'].str.contains("contrast")].index
answer=answer.drop(i)

j=answer[answer['LongCommonName'].str.contains("PET")].index
answer=answer.drop(j)

k=answer[answer['LongCommonName'].str.contains("SPECT")].index
answer=answer.drop(k)

l=answer[answer['LongCommonName'].str.contains("Functional")].index
answer=answer.drop(l)

#clean answer (move modality from LongCommonName) (replace Views from LongCommonName)
def SepModality(x):
    return x.split(' ', 1)[0]
answer['Modality']=answer['LongCommonName'].apply(SepModality)
answer['LongCommonName']=answer['LongCommonName'].str.replace(' Views', '')
answer['radiologyProtocolConceptId']=answer.radiologyProtocolConceptId.astype('int64')
answer=answer.astype(str)

#clean answer dataframe
answer['LongCommonName']=answer['LongCommonName'].str.replace('CT ', '')
answer['LongCommonName']=answer['LongCommonName'].str.replace('DXA ', '')
answer['LongCommonName']=answer['LongCommonName'].str.replace('MG ', '')
answer['LongCommonName']=answer['LongCommonName'].str.replace('MR ', '')
answer['LongCommonName']=answer['LongCommonName'].str.replace('NM ', '')
answer['LongCommonName']=answer['LongCommonName'].str.replace('PT ', '')
answer['LongCommonName']=answer['LongCommonName'].str.replace('RF ', '')
answer['LongCommonName']=answer['LongCommonName'].str.replace('US ', '')
answer['LongCommonName']=answer['LongCommonName'].str.replace('XR ', '')
answer['LongCommonName']=answer['LongCommonName'].str.replace(' bones', '')
answer['LongCommonName']=answer['LongCommonName'].str.replace(' bone', '')
answer['LongCommonName']=answer['LongCommonName'].str.replace(' body region', '')
answer['LongCommonName']=answer['LongCommonName'].str.replace(' gland', '')
answer['LongCommonName']=answer['LongCommonName'].str.replace(' - bilateral', '')
answer['LongCommonName']=answer['LongCommonName'].str.replace(' transrectal', '')

In [ ]:
def StudyDateTime(row):
    DateTime=row['StudyDate']+row['StudyTime'][0:6]
    try:
        return str(datetime.strptime(DateTime, '%Y%m%d%H%M%S'))
    except:
        try:
            return str(datetime.strptime(DateTime, '%Y%m%d'))
        except:
            return None
        
def StudyDate(row):
    try:
        return datetime.strptime(row['StudyDate'],'%Y%m%d').date().strftime('%Y-%m-%d')
    except:
        return None

def modality(x):
    if x=='CR' or x=='DX':
        return 'XR'
    elif x=='BM' or x=='DXA':
        return 'DXA'
    elif x=='PR':
        return 'MR'
    elif x=='CT' or x=='MG' or x=='MR' or x=='NM' or x=='PT' or x=='RF' or x=='US':
        return x
    else:
        return 'OT'

def lower(x):
    return x.lower()

def mapping(row):
    try:
        mapvalue=[]
        for i in answer['LongCommonName']:
            if i in row['information']:
                mapvalue.append(answer.loc[(answer['LongCommonName'] == i) & (answer['Modality']==row['newModality'])].reset_index()['radiologyProtocolConceptId'][0])
                return ' ,'.join(mapvalue)
    except:
        return None

In [ ]:
cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER=128.1.99.58;DATABASE=ID_XYLITOL_WHITE;UID=hihipch;PWD=gks1dls!')
cursor = cnxn.cursor()

In [ ]:
a = 'select * from [pacsAnonData].[dbo].[AnonData2] where StudyInstanceUID in (select studyinstanceUID from (select ROW_NUMBER() over(order by StudyInstanceUID) as row, StudyInstanceUID from (SELECT distinct top(9999999) StudyInstanceUID FROM [pacsAnonData].[dbo].[AnonData2] order by StudyInstanceUID) v) v where row between first and second)'
for i in range(19109):
    try:
        SQL_Query = pd.read_sql_query(a.replace('first', str(1+i*100)).replace('second', str(100+i*100)), cnxn)
            #Making RadiologyImageTable from SQLquery
        RadiologyImageTable=SQL_Query[['Filepath', 'ImageID', 'SeriesID', 'StudyInstanceUID', 'ImageResolutionRows', 'ImageResolutionColumns', 'SliceThickness', 'SeriesDescription']]
        RadiologyImageTable.columns=['file_path', 'image_id', 'series_id', 'study_id', 'image_resolution_rows', 'image_resolution_columns', 'CT_slice_thickness', 'series_description']
        RadiologyImageTable.loc[RadiologyImageTable['CT_slice_thickness'] == 'NA', ['CT_slice_thickness']] = None
        RadiologyImageTable.loc[RadiologyImageTable['image_resolution_rows'] == 'NA', ['image_resolution_rows']] = None
        RadiologyImageTable.loc[RadiologyImageTable['image_resolution_columns'] == 'NA', ['image_resolution_columns']] = None
        RadiologyImageTable.loc[RadiologyImageTable['series_description'] == '', ['series_description']] = None
        RadiologyImageTable.loc[RadiologyImageTable['series_description'] == 'NA', ['series_description']] = None
        RadiologyImageTable["series_type"] = None
        RadiologyImageTable["anatomical_plane"] = None
        grouped=RadiologyImageTable[['image_id','series_id']].groupby('series_id')
        PhaseTotalNo=grouped.count()
        PhaseTotalNo.columns = ['series_total_number']
        RadiologyImageTable['series_serial_number']=RadiologyImageTable.groupby(['series_id']).cumcount()+1
        RadiologyImageTable=pd.merge(RadiologyImageTable, PhaseTotalNo, on='series_id')
        RadiologyImageTable=RadiologyImageTable[['file_path', 'image_resolution_rows', 'image_resolution_columns', 'CT_slice_thickness', 'series_type', 'anatomical_plane', 'series_serial_number', 'series_total_number', 'image_id', 'series_id', 'study_id', 'series_description']]
        for index,row in RadiologyImageTable.iterrows():
            try:
                cursor.execute("insert into RADIOLOGY_IMAGE(file_path, image_resolution_rows, image_resolution_columns, CT_slice_thickness, series_type, anatomical_plane, series_serial_number, series_total_number, image_id, series_id, study_id, series_description) values (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)", row['file_path'], row['image_resolution_rows'], row['image_resolution_columns'], row['CT_slice_thickness'], row['series_type'], row['anatomical_plane'], row['series_serial_number'], row['series_total_number'], row['image_id'], row['series_id'], row['study_id'], row['series_description'])
                cnxn.commit()
            except:
                print(row)


            #Making RadiologyOccurrenceTable from SQL_Query
        RadiologyOccurrenceTable=SQL_Query[['StudyInstanceUID', 'StudyDate', 'StudyTime', 'PatientID', 'Manufacturer', 'Modality', 'BodyPartExamined', 'ProtocolName', 'StudyDescription', 'PerformedProcedureStepDescription']]
        RadiologyOccurrenceTable['Radiology_Comment']=None
        imageTotalNo=RadiologyOccurrenceTable[['StudyInstanceUID', 'PatientID']].groupby('StudyInstanceUID').count()
        imageTotalNo.columns = ['imageTotalNo']
        RadiologyOccurrenceTable=RadiologyOccurrenceTable.groupby('StudyInstanceUID').first().reset_index()
        RadiologyOccurrenceTable=pd.merge(RadiologyOccurrenceTable, imageTotalNo, on='StudyInstanceUID')

        RadiologyOccurrenceTable['StudyDateTime']=RadiologyOccurrenceTable.apply(StudyDateTime, axis=1)
        RadiologyOccurrenceTable['StudyDate']=RadiologyOccurrenceTable.apply(StudyDate, axis=1)
        del RadiologyOccurrenceTable['StudyTime']

        RadiologyOccurrenceTable['newModality']=RadiologyOccurrenceTable['Modality'].apply(modality)

        RadiologyOccurrenceTable['information']=RadiologyOccurrenceTable['ProtocolName']+","+RadiologyOccurrenceTable['PerformedProcedureStepDescription']+","+RadiologyOccurrenceTable['BodyPartExamined']+","+RadiologyOccurrenceTable['StudyDescription']

        #lowercase answerLongCommonName, RadiologyOccurrenceTableInformation
        RadiologyOccurrenceTable['information']=RadiologyOccurrenceTable['information'].apply(lower)
        answer['LongCommonName']=answer['LongCommonName'].apply(lower)

        #modify data information (c-spine : cervical spine) (remove joint, bone)
        RadiologyOccurrenceTable['information']=RadiologyOccurrenceTable['information'].str.replace('c-spine', 'cervical spine')
        RadiologyOccurrenceTable['information']=RadiologyOccurrenceTable['information'].str.replace('cervical', 'cervical spine')
        RadiologyOccurrenceTable['information']=RadiologyOccurrenceTable['information'].str.replace('l-spine', 'lumbar spine')
        RadiologyOccurrenceTable['information']=RadiologyOccurrenceTable['information'].str.replace('l -spine', 'lumbar spine')
        RadiologyOccurrenceTable['information']=RadiologyOccurrenceTable['information'].str.replace('l - spine', 'lumbar spine')
        RadiologyOccurrenceTable['information']=RadiologyOccurrenceTable['information'].str.replace('lumbar', 'lumbar spine')
        RadiologyOccurrenceTable['information']=RadiologyOccurrenceTable['information'].str.replace('lumber', 'lumbar spine')
        RadiologyOccurrenceTable['information']=RadiologyOccurrenceTable['information'].str.replace('pelvic', 'pelvis')
        RadiologyOccurrenceTable['information']=RadiologyOccurrenceTable['information'].str.replace('abd', 'abdomen')
        RadiologyOccurrenceTable['information']=RadiologyOccurrenceTable['information'].str.replace('wris', 'wrist')
        RadiologyOccurrenceTable['information']=RadiologyOccurrenceTable['information'].str.replace('joint', '')
        RadiologyOccurrenceTable['information']=RadiologyOccurrenceTable['information'].str.replace('bone', '')
        RadiologyOccurrenceTable['information']=RadiologyOccurrenceTable['information'].str.replace('pns', 'sinuses')
        RadiologyOccurrenceTable['information']=RadiologyOccurrenceTable['information'].str.replace('pancrea', 'pancreas')
        RadiologyOccurrenceTable['information']=RadiologyOccurrenceTable['information'].str.replace('l-s-spine', 'lumbar spine sacrum')
        RadiologyOccurrenceTable['information']=RadiologyOccurrenceTable['information'].str.replace('l_s spine', 'lumbar spine sacrum')
        RadiologyOccurrenceTable['information']=RadiologyOccurrenceTable['information'].str.replace('l-s spine', 'lumbar spine sacrum')
        RadiologyOccurrenceTable['information']=RadiologyOccurrenceTable['information'].str.replace('total body', 'whole body')
        RadiologyOccurrenceTable['information']=RadiologyOccurrenceTable['information'].str.replace('renal', 'kidney')
        RadiologyOccurrenceTable['information']=RadiologyOccurrenceTable['information'].str.replace('mammo', 'breast')

        RadiologyOccurrenceTable['Protocol_Concept_ID']=RadiologyOccurrenceTable.apply(mapping, axis=1)
        RadiologyOccurrenceTable=RadiologyOccurrenceTable[['StudyInstanceUID', 'StudyDate', 'StudyDateTime', 'PatientID', 'Manufacturer', 'Modality', 'Radiology_Comment', 'imageTotalNo', 'Protocol_Concept_ID', 'information']]
        RadiologyOccurrenceTable.columns=['study_id', 'study_date', 'study_datetime', 'person_id', 'manufacturer', 'modality', 'radiology_comment', 'image_total_number', 'protocol_concept_id', 'information']
        RadiologyOccurrenceTable=RadiologyOccurrenceTable[['person_id', 'study_date', 'study_datetime', 'modality', 'protocol_concept_id', 'image_total_number', 'radiology_comment', 'study_id', 'manufacturer', 'information']]
        RadiologyOccurrenceTable.loc[RadiologyOccurrenceTable['manufacturer'] == '', ['manufacturer']] = None
        for index,row in RadiologyOccurrenceTable.iterrows():
            try:
                cursor.execute("insert into RADIOLOGY_OCCURRENCE(person_id, study_date, study_datetime, modality, protocol_concept_id, image_total_number, radiology_comment, study_id, manufacturer, information) values (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)", row['person_id'], row['study_date'], row['study_datetime'], row['modality'], row['protocol_concept_id'], row['image_total_number'], row['radiology_comment'], row['study_id'], row['manufacturer'], row['information'])
                cnxn.commit()
            except:
                print(row)
    except:
        print(i)